# Data cleaning and AR for dynamic programming exam

In [20]:
#Import packages

import pandas as pd
import os
import patsy                           # provides a syntax for specifying models  
import statsmodels.api as sm           # provides statistical models like ols, gmm, anova, etc...
import statsmodels.formula.api as smf  # provides a way to directly spec models from formulas
import numpy as np

In [21]:
#Load data
file_directory = os.getcwd() + '\\Data\\' + 'sparadata.xls' 
data = pd.read_excel(file_directory, skipfooter = 3) # Remove last 3 rows that are NA 

In [22]:
# New data set for structural estimaiton. We remove observations where individuals retired before the age of 60
data_structural = data.copy()
data_structural["invalid"] = ((data_structural["ret"] == 1) & (data_structural["age"] < 60))

In [23]:
# Obs to filter out
filter_cond = data_structural[(data_structural['invalid'] == True).groupby(data_structural['id']).transform('any')] 

In [24]:
keys = list(filter_cond.columns.values)
i1 = data_structural.set_index(keys).index
i2 = filter_cond.set_index(keys).index
data_structural = data_structural[~i1.isin(i2)]

In [25]:
# Make data compatible with the solution of the model with all 5 state variables
data_structural["ret_age"] = data_structural["id"].map(data_structural[data_structural["ret"] == 1].set_index("id")["age"])
data_structural = data_structural.dropna()

In [26]:
# First step regression for income 
data['income_lead'] = data.groupby(["id"])["income"].shift(-1) # Create new column with wage tomorrow for each person
data = data.dropna() # Drop last row for each person with no observaiton for tomorrow's wage 

In [27]:
# Transform data for the reduced form AR(1) model - ignore error data is correct 
data["log_income"] = np.log(data["income"])
data["log_income_lead"] = np.log(data["income_lead"])
data["age_squared"] = data["age"] * data["age"]

In [28]:
# Income first step regression 
model = smf.ols('log_income_lead ~ log_income + age + age_squared', data = data)

results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        log_income_lead   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                 6.590e+04
Date:                Thu, 02 Jun 2022   Prob (F-statistic):               0.00
Time:                        13:48:10   Log-Likelihood:                 18133.
No. Observations:               46734   AIC:                        -3.626e+04
Df Residuals:                   46730   BIC:                        -3.622e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.2376      0.153      1.551      

In [29]:
# Variance of the error term 
results.scale

0.026949281380785624

In [30]:
# First step regression for ATP points 

# Load data again because we removed last row for each person before when we did the regression for income 
data = pd.read_excel(file_directory, skipfooter = 3) # Remove last 3 rows that are NA 

data['atp_lead'] = data.groupby(["id"])["atp"].shift(-1) # Create new column with next period ATP points 
data.drop(data[data.atp == 0].index, inplace = True) # Drop rows with ATP points = 0. We will take the log later
data = data.dropna() # Drop last row for each person with no observaiton for tomorrow's ATP points  

# Transform data for the regression
data["log_atp"] = np.log(data["atp"])
data["log_atp_lead"] = np.log(data["atp_lead"])
data["age_squared"] = data["age"] * data["age"]

In [31]:
# ATP points first step regression 
model_atp = smf.ols('log_atp_lead ~ log_atp + age + age_squared', data = data)

results_atp = model_atp.fit()
print(results_atp.summary())

                            OLS Regression Results                            
Dep. Variable:           log_atp_lead   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 4.701e+06
Date:                Thu, 02 Jun 2022   Prob (F-statistic):               0.00
Time:                        13:48:11   Log-Likelihood:             1.0493e+05
No. Observations:               46733   AIC:                        -2.099e+05
Df Residuals:                   46729   BIC:                        -2.098e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0428      0.024     -1.791      

In [32]:
# Variance of the error term 
results_atp.scale

0.0006565512692947586

In [33]:
# Transition probabilities for marital status 

# Load data again to get the last row we removed before in the first step regression
file_directory = os.getcwd() + '\\Data\\' + 'sparadata.xls' 
data = pd.read_excel(file_directory, skipfooter = 3) # Remove last 3 rows that are NA 

# Create two data sets
data_m = data.copy()
data_u = data.copy()

# Probability of married today and divorced tomorrow (and probability of married today and married tomorrow)
data_m["married_next"] = data_m.groupby(["id"])["married"].shift(-1)
data_m.dropna(inplace = True)

# = 1 if married today and divorced tomorrow, = 0 if not married today or tomorrow, = - 1 if not married today but married 
# tomorrow, = 0 if married today and tomorrow. We want only married today + tomorrow and married today + divorced tomorrow
data_m["m-unm"] = data_m["married"] - data_m["married_next"]

# = 2 if married today and married tomorrow, = 1 if married today and divorced tomorrow, = 1 if unmarried today but married
# tomorrow, = 0 if not married today or tomorrow. 
data_m["um-um"] = data_m["married"] + data_m["married_next"]

# Drop relevant rows so that column "m-unm" is = 1 if ONLY married today + divorced tomorrow and = 0 if ONLY married today
# + married tomorrow 
data_m.drop(data_m[data_m["m-unm"] == -1].index, inplace = True)
data_m.drop(data_m[data_m["um-um"] == 0].index, inplace = True)

In [34]:
# Compute probability of being married today and divorced tomorrow 
married_to_unmarried = data_m["m-unm"].sum()
observations = data_m.shape[0]

married_to_unmarried_prob = married_to_unmarried / observations * 100 
married_to_married_prob = 100 - married_to_unmarried_prob

print("The probability of being married today and divorced tomorrow is {:1.2f}%".format(married_to_unmarried_prob))
print("The probability of being married today and married tomorrow is {:1.2f}%".format(married_to_married_prob))



The probability of being married today and divorced tomorrow is 1.26%
The probability of being married today and married tomorrow is 98.74%


In [35]:
# Transition probabilities for marital status #2

# Probability of unmarried today and married tomorrow (and unmarried today and unmarried tomorrow)
data_u["married_next"] = data_u.groupby(["id"])["married"].shift(-1)
data_u.dropna(inplace = True)

# = 2 if married today and tomorrow, = 1 if married today and divorced tomorrow, = 1 if unmarried today and married tomorrow
# = 0 if unmarried today and unmarried tomorrow. We want only unmarried today + married tomorrow and unmarried today + 
# unmarried tomorrow. 
data_u["unm-m"] = data_u["married"] + data_u["married_next"] 

# = -1 if married today and divorced tomorrow
data_u["m-unm"] = data_u["married_next"] - data_u["married"]

# Drop relevant rows so that column "unm-m" is either = 0 or 1 and = 1 if ONLY unmarried today and married tomorrow
data_u.drop(data_u[data_u["unm-m"] == 2].index, inplace = True) 
data_u.drop(data_u[data_u["m-unm"] == -1].index, inplace = True)

In [36]:
# Compute probability of being unmarried today and married tomorrow 

unmarried_to_married = data_u["unm-m"].sum()
observations_unmarried_to_married = data_u.shape[0]

unmarried_to_married_prob = unmarried_to_married / observations_unmarried_to_married * 100
unmarried_to_unmarried_prob = 100 - unmarried_to_married_prob

print("The probability of being unmarried today and married tomorrow is {:1.2f}%".format(unmarried_to_married_prob))
print("The probability of being unmarried today and unmarried tomorrow is {:1.2f}%".format(unmarried_to_unmarried_prob))



The probability of being unmarried today and married tomorrow is 1.36%
The probability of being unmarried today and unmarried tomorrow is 98.64%


In [39]:
# Survival Data cleaning 
file_directory = os.getcwd() + '\\Data\\' + 'surv.xls' 
data_surv = pd.read_excel(file_directory, skipfooter = 7, skiprows = range(1,21), header = 0)

In [40]:
# CCP for sample (Hazard rates)
data["work"] = 1 - data.ret # = 1 if working, = 0 if retired
working = data.groupby(["age"])["work"].sum()
retiring = data.groupby(["age"])["ret"].sum()
hazard_rates = retiring / (working + retiring)
CCP_sample = np.ones(np.shape(hazard_rates)) - hazard_rates
CCP_sample = CCP_sample[8:] # get correct age
np.cumprod(CCP_sample) # Share of population working at given age

age
58    0.995535
59    0.986674
60    0.955882
61    0.930644
62    0.899113
63    0.855077
64    0.694113
65    0.224149
66    0.171301
67    0.140372
68    0.112297
69    0.102939
70    0.102939
dtype: float64

In [59]:
hazard_rates

age
50    0.000000
51    0.000560
52    0.000264
53    0.001254
54    0.001194
55    0.000908
56    0.002381
57    0.004557
58    0.004465
59    0.008900
60    0.031208
61    0.026403
62    0.033881
63    0.048977
64    0.188245
65    0.677071
66    0.235772
67    0.180556
68    0.200000
69    0.083333
70    0.000000
dtype: float64

In [41]:
# Expected retirement age (weighted average)
sum(np.arange(50,71) * np.transpose(hazard_rates)) / hazard_rates.sum()

65.39865657449695